# Pandas for Data Analysis: Extra Useful Stuff

## Outline:

* [Fetching Data from APIs](#Fetching-Data-from-APIs)
* [Web Scraping to Build Your Own Dataset](#Web-Scraping-to-Build-Your-Own-Dataset)
* [Anonymizing Data with Faker](#Anonymizing-Data-with-Faker)
* [Ploting Data on Google Maps with gmaps](#Plotting-Data-on-Google-Maps-with-gmaps)

## Fetching Data from APIs

[JSONPlaceholder](https://jsonplaceholder.typicode.com/) comes with a set of 6 common resources:


| Endpoint  | Data         |
|-----------|--------------|
| /posts    | 100 posts    |
| /comments | 500 comments |
| /albums   | 100 albums   |
| /photos   | 5000 photos  |
| /todos    | 200 todos    |
| /users    | 10 users     |

In [ ]:
import pandas as pd
import requests

In [ ]:
users_url = 'https://jsonplaceholder.typicode.com/users'
r = requests.get(users_url)
data = r.json()
data

In [ ]:
ids = [each['id'] for each in data]
usernames = [each['username'] for each in data]

In [ ]:
user_df = pd.DataFrame(data={
    'id': ids,
    'username': usernames
})

In [ ]:
user_df.head()

In [ ]:
albums_url = 'https://jsonplaceholder.typicode.com/albums'
r = requests.get(albums_url)
data = r.json()
data

In [ ]:
album_df = pd.DataFrame(data=data)

In [ ]:
album_df.head()

In [ ]:
users_albums = pd.merge(user_df, album_df, how='inner', left_on='id', right_on='userId')

In [ ]:
users_albums

In [ ]:
users_albums.username.value_counts()

## Web Scraping to Build Your Own Dataset

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
url = 'http://econpy.pythonanywhere.com/ex/001.html'
r = requests.get(url)

In [ ]:
soup = BeautifulSoup(r.content, 'lxml')
soup

In [ ]:
soup.title

In [ ]:
soup.div

In [ ]:
soup.find_all('div', title='buyer-name')

In [ ]:
names = []
for each in soup.find_all('div', title='buyer-name'):
    names.append(each.string)
    
names

In [ ]:
soup.find_all(attrs={'class': 'item-price'})

In [ ]:
soup.find_all('span', class_='item-price')

In [ ]:
prices = []
for each in soup.find_all(attrs={'class': 'item-price'}):
    prices.append(each.string)
                  
prices

In [ ]:
list(zip(names, prices))

In [ ]:
column_headers = ['name', 'price']
df = pd.DataFrame(list(zip(names, prices)), columns=column_headers)

In [ ]:
df.head()

เนื่องจากเว็บอาจจะมีการใช้ pagination เพื่อให้การแสดงผลข้อมูลแบ่งออกเป็นแต่ละหน้า ทำให้ข้อมูลไม่อยู่ในหน้าเดียว วิธีแก้ปัญหาคือ เราจะไล่ scrape ทุกหน้าไปเรื่อยๆ

In [ ]:
pages = ['001', '002', '003', '004', '005']
names = []
prices = []
for each_page in pages:
    url = 'http://econpy.pythonanywhere.com/ex/' + each_page + '.html'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')
    
    for each in soup.find_all('div', title='buyer-name'):
        names.append(each.string)

    for each in soup.find_all(attrs={'class': 'item-price'}):
        prices.append(each.string)

In [ ]:
column_headers = ['name', 'price']
df = pd.DataFrame(list(zip(names, prices)), columns=column_headers)

In [ ]:
df.head()

## Anonymizing Data with Faker

In [ ]:
import pandas as pd
from faker import Faker

In [ ]:
people_data_url = 'https://raw.githubusercontent.com/lawlesst/vivo-sample-data/master/data/csv/people.csv' 
df = pd.read_csv(people_data_url)
df.head()

In [ ]:
fake = Faker()

In [ ]:
fake.name()

In [ ]:
fake.email()

In [ ]:
def new_name(name):
    return fake.name()

In [ ]:
df['new_name'] = df.name.map(new_name)

In [ ]:
df.head()

## Plotting Data on Google Maps with gmaps

gmaps: https://jupyter-gmaps.readthedocs.io/en/latest/

In [ ]:
import gmaps
import gmaps.datasets

ถ้าต้องการให้การแสดงผลข้อมูลในแผนที่เราถูกต้องมากขึ้น และใช้งานได้เยอะๆ เราควรต้องใช้ API key จาก Google

In [ ]:
#gmaps.configure(api_key="AI...") # Your Google API key

In [ ]:
locations = gmaps.datasets.load_dataset('taxi_rides')

In [ ]:
locations

In [ ]:
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations))
fig

In [ ]:
earthquake_df = gmaps.datasets.load_dataset_as_df('earthquakes')
earthquake_df.head()

In [ ]:
locations = earthquake_df[['latitude', 'longitude']]
weights = earthquake_df['magnitude']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig